In [35]:
from typing import Dict, List
from datamodel import Order, OrderDepth, TradingState, Trade
import json
import math

class Trader:
    def __init__(self):
        self.mid_prices = {
            "CROISSANTS": [],
            "JAMS": [],
            "DJEMBES": [],
            "PICNIC_BASKET1": []
        }

    # Position limits (maximum exposure) for the instruments.
    POSITION_LIMITS = {
        "CROISSANTS": 250,
        "JAMS": 350,
        "PICNIC_BASKET2": 100
    }
    
    TRADE_SIZE = 1         # We always trade 1 unit per signal.
    MIN_SPREAD = 1.0       # Only trade if the spread is at least 1 seashell.

    def run(self, state: TradingState):
        orders = {}
        conversions = 0
        instruments = ["CROISSANTS", "JAMS", "PICNIC_BASKET2"]

        for prod in instruments:
            if prod not in state.order_depths:
                continue

            od = state.order_depths[prod]
            best_bid, bid_vol = self.get_best_bid(od)
            best_ask, ask_vol = self.get_best_ask(od)
            if best_bid is None or best_ask is None:
                print(f"{prod}: Incomplete order book, skipping.")
                continue

            spread = best_ask - best_bid
            if spread < self.MIN_SPREAD:
                print(f"{prod}: Spread {spread:.2f} is below MIN_SPREAD {self.MIN_SPREAD}, skipping trade.")
                continue

            current_pos = state.position.get(prod, 0)
            print(f"{prod} — Best Bid: {best_bid} (vol {bid_vol}), Best Ask: {best_ask} (vol {ask_vol}), Current Position: {current_pos}")

            # If flat: post both a buy and sell order.
            if current_pos == 0:
                if bid_vol >= self.TRADE_SIZE:
                    print(f"{prod}: Flat. Posting BUY order for {self.TRADE_SIZE} unit at {best_bid}.")
                    orders.setdefault(prod, []).append(Order(prod, best_bid, self.TRADE_SIZE))
                else:
                    print(f"{prod}: Not enough bid volume for BUY.")
                if ask_vol >= self.TRADE_SIZE:
                    print(f"{prod}: Flat. Posting SELL order for {self.TRADE_SIZE} unit at {best_ask}.")
                    orders.setdefault(prod, []).append(Order(prod, best_ask, -self.TRADE_SIZE))
                else:
                    print(f"{prod}: Not enough ask volume for SELL.")
            # When long: only post sell orders (to reduce long exposure).
            elif current_pos > 0:
                if ask_vol >= self.TRADE_SIZE:
                    print(f"{prod}: Long ({current_pos}). Posting SELL order for {self.TRADE_SIZE} unit at {best_ask}.")
                    orders.setdefault(prod, []).append(Order(prod, best_ask, -self.TRADE_SIZE))
                else:
                    print(f"{prod}: Not enough ask volume to sell from long position.")
            # When short: only post buy orders (to reduce short exposure).
            elif current_pos < 0:
                if bid_vol >= self.TRADE_SIZE:
                    print(f"{prod}: Short ({current_pos}). Posting BUY order for {self.TRADE_SIZE} unit at {best_bid}.")
                    orders.setdefault(prod, []).append(Order(prod, best_bid, self.TRADE_SIZE))
                else:
                    print(f"{prod}: Not enough bid volume to cover short position.")

        print("Final orders:", orders)
        # No extra state is used in this simple strategy.
        return orders, conversions, "simple_profit_v1"

    def get_best_bid(self, order_depth: OrderDepth):
        """Return the highest bid price and its associated volume from the order depth."""
        if order_depth.buy_orders:
            best_bid = max(order_depth.buy_orders.keys())
            return best_bid, order_depth.buy_orders[best_bid]
        return None, None

    def get_best_ask(self, order_depth: OrderDepth):
        """Return the lowest ask price and its associated volume from the order depth."""
        if order_depth.sell_orders:
            best_ask = min(order_depth.sell_orders.keys())
            return best_ask, order_depth.sell_orders[best_ask]
        return None, None

ModuleNotFoundError: No module named 'datamodel'